In [333]:
from numpy import genfromtxt
import numpy as np
def readDataAndLabels(fileName):
    train = genfromtxt(fileName, delimiter=',', dtype=float, skip_header=1)
    return (train[:, 0], train[:, 1:])

def oneHot(array):
    m = np.max(array)
    ar = np.zeros((len(array), m+1))
    y=0
    for x in array:
        ar[y, x] = 1
        y+=1
    return ar

class SigmoidActivationFunction:
    def value(self, x):
        return 1/(1 + np.exp(-x))

    def derivative(self, x):
        v = self.value(x)
        return np.multiply(v, 1-v)

class NeuralNet:
    def __init__(self, layerSizes, activationFunction):
        self._layerSizes = layerSizes
        self._activationFunction = activationFunction
        self.initRandomWeights()
        
    def initRandomWeights(self):
        self._weights = []
        for layer in range(len(self._layerSizes)-1):
            self._weights.append(np.random.randn(self._layerSizes[layer], self._layerSizes[layer+1]))
        
        self._biases = []
        for bias in range(len(self._layerSizes)-1):
            self._biases.append(np.random.randn(1, self._layerSizes[bias+1]))
            
    def getWeights(self):
        return (self._weights, self._biases)
    
    def getWeightsWithBiases(self):
        weights = list()
        for w,b in zip(self._weights, self._biases):
            weights.append(np.concatenate((b, w), axis=0))
        return weights
    
    def setWeights(self, weights, biases):
        self._weights = weights
        self._biases = biases
    
    def forward(self, X):
        zlist, aList = [],[]
        for weight, biases in zip(self._weights, self._biases):
            X = X.dot(weight) + biases
            zlist.append(X)
            X = self._activationFunction.value(X)
            aList.append(X)
        return (X, zlist, aList)
    
    def addOnesForBiasCalculation(self, X):
        z = np.ones((len(X),1))
        return np.concatenate((z, X), axis=1)

    
    def backpropagation(self, X, Y, costFunction, learningRate = 0.0001):
        pred, zList, aList = self.forward(X)
        error = costFunction.cost(pred, Y)
        print(error)
        zList = zList[::-1]
        aList = aList[::-1]

        delta3 = np.multiply(-(Y-pred), self._activationFunction.derivative(zList[0]))
        dJdW2 = np.dot(aList[1].T, delta3)
        
        delta2 = np.dot(delta3, self._weights[1].T)*self._activationFunction.derivative(zList[1])
        dJdW1 = np.dot(X.T, delta2)
        
        self._weights[1] -= learningRate * dJdW2
        self._weights[0] -= learningRate * dJdW1
        
        
        print(self._biases[1].shape, delta3.shape)
        print(self._biases[0].shape, delta2.shape)
        #self._biases[1] -= learningRate * np.mean(delta2, 0)
        #self._biases[0] -= learningRate * np.mean(delta1, 0)
        #print(self._weights[1].shape, dJdW2.shape)
        #print(self._weights[0].shape, dJdW1.shape)
        #return dJdW1, dJdW2
        



\begin{align}
J
&= \left[ \frac{1}{m} \sum_{i=1}^m \left( \frac{1}{2} \left\| h_{W,b}(x^{(i)}) - y^{(i)} \right\|^2 \right) \right]
                       + \frac{\lambda}{2} \sum_{l=1}^{n_l-1} \; \sum_{i=1}^{s_l} \; \sum_{j=1}^{s_{l+1}} \left( W^{(l)}_{ji} \right)^2
\end{align}


In [257]:
class MSECost:
    def cost(self, predicted, actual):
        return 0.5 * np.sum(np.power(predicted - actual, 2))/len(predicted)

In [100]:
datalabels, datapixels = readDataAndLabels('Data/train.csv')

In [331]:
neuralNetwork = NeuralNet([784, 25, 10], SigmoidActivationFunction())
pr, zList, aList = neuralNetwork.forward(datapixels/255)
actual = oneHot(datalabels.astype(int))

weights, biases = neuralNetwork.getWeights()

for x in weights:
    print(x.shape)

for x in biases:
    print(x.shape)
    

#for x in range(30):
#    neuralNetwork.backpropagation(datapixels/255, actual, MSECost(), 0.0001)


(784, 25)
(25, 10)
(1, 25)
(1, 10)


array([[ 1.        ,  0.75016627,  0.87476407],
       [ 1.        ,  0.73669149,  0.64655557],
       [ 1.        ,  0.73464359,  0.12799447]])

In [129]:
#class Network(object):
#
#    def __init__(self, sizes):
#        self.num_layers = len(sizes)
#        self.sizes = sizes
#        self.large_weight_initializer()
#
#    def large_weight_initializer(self):
#        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
#        self.weights = [np.random.randn(y, x)
#                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]
#    def feedforward(self, a):
#        for b, w in zip(self.biases, self.weights):
#            a = np.dot(w, a)
#            print(a.shape)
#        return a
#        
#n = Network([784, 25, 10])
#n.feedforward(datapixels.transpose())


255.0

In [170]:
#oneHot(np.array([0,4,3,4,2,1,1]))

In [319]:
A = np.random.rand(3,2)
B = np.random.rand(3,2)


b = np.random.rand(1,2)

AttributeError: 'NeuralNet' object has no attribute 'addOnesForBiasCalculation'

In [180]:
B

array([[ 0.97621369,  0.6617447 ],
       [ 0.90703541,  0.88849807],
       [ 0.20557865,  0.38961472]])

In [192]:
np.sum((A-B)**2)/len(A)

0.39071981001258638

In [190]:
np.mean(np.sum((A-B)**2, 1))

0.39071981001258638